# <img src="../assets/logo_infnetv1.png" alt="Infnet logo" height="45"/> Pipeline para o projeto de Disciplina de Validação de Modelos de Clusterização
<img src="https://img.shields.io/badge/python-v._3.11.5-blue?style=flat-square&logo=python&logoColor=white" alt="python_logo" height="20"/>
<img src="https://img.shields.io/badge/jupyter-v._5.7.2-blue?style=flat-square&logo=jupyter&logoColor=white" alt="jupyter_logo" height="20"/>
<img src="https://img.shields.io/badge/anaconda-v._23.7.4-blue?style=flat-square&logo=anaconda&logoColor=white" alt="anaconda_logo" height="20"/>

#### Aluno: Mateus Teixeira Ramos da Silva

### Índice

---

- <a href='#importar-o-arquivo-não-tratado'>Importar o arquivo não tratado</a>

- <a href='#colunas-temporais-idade-data-da-transação-e-time_stamp'>Colunas temporais</a>

- <a href='#colunas-categóricas'>Colunas categóricas</a>

- <a href='#colunas-numéricas'>Colunas numéricas</a>

- <a href='#salvar-o-arquivo-tratado'>Salvar o arquivo tratado</a>


### Objetivo:

Tratar os dados do dataset que se encontram fora de padrão

In [79]:
# Importar os pacotes

# Para demonstrar a infraestrutura utilizada
import sys
import subprocess
import os
import pkg_resources

# Para administrar o dataset
import numpy as np
import pandas as pd

# Para preprocessamento dos dados
from datetime import datetime
from sklearn.preprocessing import RobustScaler

#### Importar o arquivo não tratado

---

<a href='#índice'>Voltar ao início</a>

In [80]:
caminho = '..\data\\'
arquivo = 'bank_transactions' + '.csv'

# Caminho completo para o novo arquivo
caminho_arquivo = os.path.join(caminho, arquivo)

base = pd.read_csv(caminho_arquivo, encoding='latin-1', on_bad_lines='skip', sep=',')

In [81]:
# Definindo uma amostra da base para determinar o número ótimo de K
fracao = 0.015 # para 2.5% 10.000 da base
dados = base.sample(frac=fracao, random_state=6)

In [82]:
dados.shape

(15729, 9)

In [83]:
# Traduzir as colunas
colunas_traduzidas = ['id_transacao', 'id_cliente', 'idade', 'genero', 'localizacao', 'saldo', 'data_transacao', 'time_stamp', 'quantia_transacao (INR)']

dados.columns = colunas_traduzidas

In [84]:
# Detalhando a base de dados com moda
def check(df):
    l = []
    colunas = df.columns
    
    for col in colunas:
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        sum_null = df[col].isnull().sum()

        # Calcula a moda e a frequência da moda
        moda = df[col].mode().iloc[0] if not df[col].mode().empty else "Não se aplica"
        moda_freq = df[col].value_counts().iloc[0] if not df[col].value_counts().empty else "Não se aplica"

        if np.issubdtype(dtypes, np.number):
            status = df.describe(include='all').T
            media = status.loc[col, 'mean']
            std = status.loc[col, 'std']
            min_val = status.loc[col, 'min']
            quar1 = status.loc[col, '25%']
            mediana = df[col].median()
            quar3 = status.loc[col, '75%']
            max_val = status.loc[col, 'max']
                    
        else:
            status = "Não se aplica"
            media = "Não se aplica"
            std = "Não se aplica"
            min_val = "Não se aplica"
            quar1 = "Não se aplica"
            mediana = "Não se aplica"
            quar3 = "Não se aplica"
            max_val = "Não se aplica"

        l.append([col, dtypes, nunique, sum_null, media, std, min_val, quar1, mediana, quar3, max_val, moda, moda_freq])
    
    # Criação do DataFrame com as novas colunas
    df_check = pd.DataFrame(l)
    df_check.columns = ['coluna', 'tipo', 'únicos', 'null_soma', 'media', 'desvio', 
                        'minimo', '25%', 'mediana', '75%', 'maximo', 'moda', 'frequência_moda']
    
    return df_check 

check(dados)

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,id_transacao,object,15729,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,T1000106,1
1,id_cliente,object,15693,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,C1028915,2
2,idade,object,7000,60,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,1/1/1800,837
3,genero,object,2,16,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,M,11581
4,localizacao,object,1644,1,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,MUMBAI,1570
5,saldo,float64,13996,35,104526.834465,457855.109357,0.0,4688.615,16609.17,56232.4425,17772978.13,0.0,46
6,data_transacao,object,54,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,6/8/16,440
7,time_stamp,int64,13793,0,156396.655159,51868.456325,0.0,123426.0,163727.0,195840.0,235959.0,161734,5
8,quantia_transacao (INR),float64,4584,0,1649.588309,8419.478006,0.0,169.0,452.0,1189.0,569500.27,100.0,518


### Colunas temporais ('idade', 'data da transação' e 'time_stamp')

---

Podemos perceber que as colunas temporais precisam de formatações e ajustes.

<a href='#índice'>Voltar ao início</a>

#### Análise exploratória

É possível verificar que as colunas 'idade' e'data_transacao' estão fora do padrão (dd/mm/yyyy)

In [85]:
dados.sample(n=3, random_state=17)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR)
980318,T980319,C3139092,25/11/59,M,BANGALORE,43804.28,15/9/16,95717,480.0
159457,T159458,C5752387,2/7/96,F,CHURACHANDPUR,39.58,4/8/16,110154,45.0
17899,T17900,C7522819,18/4/96,M,BANGALORE,2058.80,26/9/16,152339,207.0


#### Padronizar as datas

Daí a necessidade de padronizar o formato das datas para cálculo posterior

In [86]:
# Função para padronizar as datas
def padronizar_data(data):
    try:
        # Divide a data em partes
        partes = data.split('/')
        if len(partes) != 3:
            return None  # Ignorar se a estrutura estiver errada
        
        dia, mes, ano = partes

        # Adiciona zeros à esquerda no dia e mês, se necessário
        dia = dia.zfill(2)
        mes = mes.zfill(2)

        # Ajusta o ano para 4 dígitos, se necessário
        if len(ano) == 2:
            ano = '19' + ano if int(ano) > 30 else '20' + ano  # Supondo anos acima de 30 como 1900s
        
        return f"{dia}/{mes}/{ano}"
   
    except Exception as e:
        return None

# Aplica a função à coluna 'idade'
dados['idade'] = dados['idade'].apply(padronizar_data)
dados['data_transacao'] = dados['data_transacao'].apply(padronizar_data)

In [87]:
dados.sample(n=3, random_state=21)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR)
108180,T108181,C4438247,24/07/1985,M,GURGAON,19751.15,01/08/2016,230016,176.0
758475,T758476,C7915178,23/05/1978,F,RUPNAGAR,5588.98,01/09/2016,184348,94.0
143143,T143144,C7820292,01/01/1800,M,MUMBAI,737999.80,05/08/2016,142416,2000.0


#### Calcular a idade

Com as datas padronizadas é possível calcular a idade dos clientes na época das transações.

In [88]:
# Função para calcular a idade com base na data da transação
def calcular_idade_com_transacao(row):
    try:
        # Converte as strings de data para objetos datetime
        data_nascimento = datetime.strptime(row['idade'], '%d/%m/%Y')
        data_transacao = datetime.strptime(row['data_transacao'], '%d/%m/%Y')
        
        # Calcula a idade em anos com base na data da transação
        idade = data_transacao.year - data_nascimento.year
        
        # Ajusta se o aniversário ainda não aconteceu até a data da transação
        if (data_transacao.month, data_transacao.day) < (data_nascimento.month, data_nascimento.day):
            idade -= 1
        
        return idade
    
    except Exception:
        return None  # Retorna None se a data for inválida

# Aplica a função diretamente sem lambda
dados['idade_calculada'] = dados.apply(calcular_idade_com_transacao, axis=1)

Podemos perceber que temos idades inválidas (outliers) que teremos que tratar, substituindo pela média (do intervalo válido de dados)

In [89]:
check(dados[['idade_calculada']])

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,idade_calculada,float64,77,60,40.57668,42.539856,-13.0,25.0,29.0,36.0,216.0,25.0,1092


#### Isolar as idades validas

Foi escolhido o período entre 122 (idade do ser humano mais velho do mundo) e 10 (idade mínima para abertura de uma conta no banco)

*Na India, crianças a partir de 10 anos de idade podem abrir uma conta bancária em seu próprio nome em muitos bancos, mas essas contas têm restrições, como limites de transações e necessidade de supervisão parental.

In [90]:
idades_validas = dados.loc[dados['idade_calculada'] < 122].loc[dados['idade_calculada'] > 10]

media_idades_validas = round(idades_validas['idade_calculada'].mean())

media_idades_validas

31

In [92]:
# Preenche os valores nulos na coluna 'idade_calculada' com a média
dados['idade_calculada'] = dados['idade_calculada'].fillna(media_idades_validas)

In [94]:
dados.reset_index(drop=True, inplace=True)

# Substituir idades inválidas e nulas pela média
for i in range(len(dados)):
    if pd.isnull(dados.at[i, 'idade_calculada']) or dados.at[i, 'idade_calculada'] > 122 or dados.at[i, 'idade_calculada'] < 10:
        dados.at[i, 'idade_calculada'] = media_idades_validas

# Transformar a idade em 'int'
dados['idade_calculada'] = dados['idade_calculada'].astype(int)

In [96]:
dados.loc[dados['idade'].isnull()].sort_values(by='idade_calculada', ascending=False).head(3)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR),idade_calculada
227,T466341,C5041333,None,F,BENGALURU,0.00,21/08/2016,193831,1098.0,31
438,T829297,C6377342,None,F,BANGALORE,110637.14,04/09/2016,154342,2569.0,31
7946,T626687,C5426018,None,M,DELHI,177.73,29/08/2016,145612,2000.0,31


Podemos verificar que as idades estão variando de 11 a 85 anos. Todas idades consideradas legalmente válidas.

In [97]:
check(dados[['idade_calculada']])

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,idade_calculada,int32,70,0,30.708309,8.498054,12.0,25.0,29.0,34.0,84.0,31,1584


#### Coluna 'time_stamp'

Em 'time_stamp' temos a quantidade de segundos que se passaram desde a data da transação. Com isso, é possível saber a hora que a transação ocorreu.

In [98]:
# Converter a coluna 'data' para datetime
dados['data_transacao'] = pd.to_datetime(dados['data_transacao'], format='%d/%m/%Y')

# Adicionar o 'time_stamp' (segundos) à data
dados['hora_transacao'] = dados['data_transacao'] + pd.to_timedelta(dados['time_stamp'], unit='s')
dados['hora_transacao'] = dados['hora_transacao'].dt.strftime('%H:%M:%S')

In [99]:
dados.sample(n=3, random_state=21)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR),idade_calculada,hora_transacao
13733,T108181,C4438247,24/07/1985,M,GURGAON,19751.15,2016-08-01,230016,176.0,31,15:53:36
14850,T758476,C7915178,23/05/1978,F,RUPNAGAR,5588.98,2016-09-01,184348,94.0,38,03:12:28
132,T143144,C7820292,01/01/1800,M,MUMBAI,737999.80,2016-08-05,142416,2000.0,31,15:33:36


### Colunas categóricas

---

As colunas categóricas possuem muitos dados nulos que necessitam de tratamento.

<a href='#índice'>Voltar ao início</a>

#### Coluna 'gênero'

In [100]:
dados['genero'].describe()

count     15713
unique        2
top           M
freq      11581
Name: genero, dtype: object

Pode-se ver que existem dados NAN

In [101]:
dados['genero'].unique()

array(['F', 'M', nan], dtype=object)

Observa-se que possui muitos dados nulos

In [102]:
dados['genero'].isnull().sum()

16

##### Verificando a distribuição dos grupos e a moda

In [103]:
dados['genero'].value_counts()

genero
M    11581
F     4132
Name: count, dtype: int64

Para os dados NAN, optou-se por distribuí-los propocionalmente, de modo a não afetar a média

In [104]:
# Calcular a proporção dos valores 'M' e 'F' para substituição dos NaN
proporcao_M = dados['genero'].value_counts(normalize=True).get('M', 0)
proporcao_F = dados['genero'].value_counts(normalize=True).get('F', 0)

# Substituir os NaN com base na proporção
num_NA = dados['genero'].isna().sum()

# Gerar um array de substituições baseadas nas proporções
substituicoes = np.random.choice(['M', 'F'], size=num_NA, p=[proporcao_M, proporcao_F])

# Substituir os valores NaN
dados.loc[dados['genero'].isna(), 'genero'] = substituicoes

In [105]:
dados['genero'].value_counts()

genero
M    11589
F     4140
Name: count, dtype: int64

In [106]:
dados['genero'].isnull().sum()

0

#### Coluna 'localizacao'

Esta coluna possui um número grande de categorias e ainda possui dados nulos que necessitam de tratamento.

In [107]:
dados['localizacao'].describe()

count      15728
unique      1644
top       MUMBAI
freq        1570
Name: localizacao, dtype: object

In [108]:
dados['localizacao'].unique()

array(['NEW DELHI', 'GURGAON', 'KALYAN', ..., 'E G DIST (EAST GODAVARI)',
       'ALDONA', 'RR DIST'], dtype=object)

In [109]:
dados['localizacao'].isnull().sum()

1

In [110]:
dados['localizacao'].value_counts()

localizacao
MUMBAI                       1570
NEW DELHI                    1269
BANGALORE                    1220
GURGAON                      1079
DELHI                        1042
                             ... 
MULUND W MUMBAI                 1
OT CHENNAI                      1
UTTAR DINAJPUR                  1
VIRGONAGAR POST BANGALORE       1
RR DIST                         1
Name: count, Length: 1644, dtype: int64

Como são poucos dados nulos (em comparação com o tamanho do dataset), optou-se por incluí-los na moda 'MUMBAI'

In [111]:
# Substituir os valores nulos pela moda da coluna 'localizacao'
moda_localizacao = dados['localizacao'].mode()[0]  # Calcula a moda da coluna

# Substituir valores nulos (NaN) pela moda sem o inplace
dados['localizacao'] = dados['localizacao'].fillna(moda_localizacao)

# Verificar o resultado
print(dados['localizacao'].isnull().sum())  # Deve retornar 0, pois todos os NaNs foram substituídos

0


In [112]:
check(dados[['localizacao']])

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,localizacao,object,1644,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,MUMBAI,1571


### Colunas numéricas

---

Colunas como 'saldo' e 'quantia_transacao' também possuem valores nulos que necessitam de tratamento.

<a href='#índice'>Voltar ao início</a>

#### Coluna 'saldo'

É possível perceber que existem muitos dados nulos.

In [113]:
check(dados[['saldo']])

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,saldo,float64,13996,35,104526.834465,457855.109357,0.0,4688.615,16609.17,56232.4425,17772978.13,0.0,46


In [114]:
dados['saldo'].isnull().sum()

35

In [115]:
dados.sort_values(by='saldo', ascending=True).head(3)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR),idade_calculada,hora_transacao
8381,T210118,C8223079,01/01/1800,M,CHENNAI,0.0,2016-09-03,141336,2520.0,31,15:15:36
9494,T248822,C8377225,17/03/1989,F,SHIMLA,0.0,2016-08-12,93418,200.0,27,01:56:58
2953,T887035,C6828762,25/02/1995,M,CHENNAI,0.0,2016-09-07,221746,1690.0,21,13:35:46


In [116]:
dados.sort_values(by='saldo', ascending=False).head(3)

,id_transacao,id_cliente,idade,genero,localizacao,saldo,data_transacao,time_stamp,quantia_transacao (INR),idade_calculada,hora_transacao
7389,T552644,C6637939,01/01/1800,M,KOLKATA,17772978.13,2016-08-22,3509,17726.00,31,00:58:29
13111,T460541,C3837962,01/01/1800,M,KOLKATA,17772978.13,2016-08-21,150744,12116.18,31,17:52:24
6495,T765085,C3126564,27/10/1970,M,MUMBAI,15544215.36,2016-09-01,132353,331.00,45,12:45:53


O saldo varia entre 0 e 115.035.495,1 rupias indianas. Ambos os valores são válidos, portanto não temos outliers (dados nulos ou impossíveis)

Apesar de ser números válidos, se apresente com um intervalo muito grande para utilizar a média, por isso optou-se por substituir os valores NAN pela mediana dos dados válidos.

In [117]:
# Substituir valores NaN pela mediana da coluna 'saldo'
mediana_saldo = dados['saldo'].median()  # Calcula a mediana da coluna

# Substitui os NaN pela mediana
dados['saldo'] = dados['saldo'].fillna(mediana_saldo)

# Verificar se ainda há valores NaN
print(dados['saldo'].isnull().sum())  # Deve retornar 0, se todos os NaNs foram substituídos

0


Não há mais dados nulos no dataset (considerando que a coluna 'idade' foi tratada em 'idade_calculada')

In [118]:
check(dados)

,coluna,tipo,únicos,null_soma,media,desvio,minimo,25%,mediana,75%,maximo,moda,frequência_moda
0,id_transacao,object,15729,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,T1000106,1
1,id_cliente,object,15693,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,C1028915,2
2,idade,object,7000,60,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,01/01/1800,837
3,genero,object,2,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,M,11589
4,localizacao,object,1644,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,MUMBAI,1571
5,saldo,float64,13997,0,104331.201033,457364.148862,0.0,4698.99,16609.17,56051.97,17772978.13,0.0,46
6,data_transacao,datetime64[ns],54,0,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,2016-08-06 00:00:00,440
7,time_stamp,int64,13793,0,156396.655159,51868.456325,0.0,123426.0,163727.0,195840.0,235959.0,161734,5
8,quantia_transacao (INR),float64,4584,0,1649.588309,8419.478006,0.0,169.0,452.0,1189.0,569500.27,100.0,518
9,idade_calculada,int32,70,0,30.708309,8.498054,12.0,25.0,29.0,34.0,84.0,31,1584


### Salvar o arquivo tratado

---

<a href='#índice'>Voltar ao início</a>

In [119]:
# Obter o diretório onde o arquivo original está localizado
diretorio_atual = os.path.dirname(os.path.abspath('arquivo.csv'))  # Substitua 'arquivo.csv' pelo nome do arquivo original

# Caminho completo para o novo arquivo
caminho_arquivo_novo = os.path.join(caminho, 'bank_transactions_clean_data.csv')

# Salvar o DataFrame no novo arquivo
dados.to_csv(caminho_arquivo_novo, index=False)

# Verificar se o arquivo foi salvo corretamente
print(f"Arquivo salvo como '{caminho_arquivo_novo}'.")

Arquivo salvo como '..\data\bank_transactions_clean_data.csv'.
